In [1]:
def _PyLammps_end_of_step_callback(lmp=None):
    return 0

In [1]:
from lammps import PyLammps
L = PyLammps()

# --------- Seting up boxes and atoms ----------
L.units("lj")
L.dimension(2) 
L.boundary("p","p","p") #boundary conditions
L.atom_style("atomic") #type of particles, depending of the type they have their own peroperties 
#"dipole stile include charge and dipole moment"

L.region("box","block",0,30,0,30,-0.5,0.5)  #name type and sizes 
L.create_box(2,"box") #create box 

# Place atoms on a hex lattice to avoid overlaps
L.lattice("hex", 0.85)           

L.create_atoms(1,"random", 147 ,1235,"box",type,)
L.group("fluid","type",1)  #atoms of type 1 , fluid is the name

L.mass(1, 13.0) #mass atom 1

L.create_atoms(2, "single",13.5,7,0) #create only one atom 
L.group("center","type",2) 
L.mass(2, 10.0)


# --------- Interactions & neighbors ---------- it uses the lj potential 
L.pair_style("lj/cut", 2.5) 

sig12 = 5          # bigger "size" -> they get expelled sooner
eps12 = 60.0          # stronger repulsion
rc12  = 1.1224620483 * sig12
L.pair_coeff(1, 1, 1.0, 1.0, 2.5)   # fluid-fluid
L.pair_coeff(1, 2, eps12, sig12, rc12)# fluid- atom stronger repulsion
L.pair_coeff(2, 2, 1.0, 1.0, 2.5) 
L.neighbor(0.3, "bin")
L.neigh_modify("delay", 0, "every", 1, "check", "yes")  # update every step

# --------- Velocities ----------
L.velocity("fluid", "create", 0.8, 87287, "mom", "yes", "rot", "no") #initial velcoity of the fluid
L.velocity("center","set",0,0,0)
L.fix("hold_center", "center", "move", "linear", 0.0, 0.0, 0.0) #fixes the atom in the center

# --------- Output ----------
L.thermo(100);
L.thermo_style("custom","step","temp","pe","ke","etotal","press","density");
L.dump(1,"all","atom",200,"traj_2d_lj.lammpstrj");
L.dump_modify(1,"sort","id");

# --------- warm-up to avoid blow-ups ----------
L.timestep(0.002)                 # smaller, stable dt
L.fix(10, "all", "nve/limit", 0.1) # caps per-step displacement
L.command("run 2000");
L.unfix(10)

# --------- Main thermostat + integrate ----------
L.fix(1, "all", "langevin", 1.0, 1.0, 1.0, 904297)  # hold T=1
L.fix(2, "all", "nve")
L.fix(3, "all", "enforce2d")
L.command("run 40000");


ModuleNotFoundError: No module named 'lammps'